## Naive Retrieval-Augmented Generation

источники:
- [Примеры реализации RAG](https://github.com/yandex-datasphere/advanced_rag/tree/main)

In [1]:
%pip install -q langchain==0.2.8 langchain-chroma==0.1.2 chromadb==0.4.18
%pip install -q --no-deps yandex-chain==0.0.9

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from yandex_chain import YandexEmbeddings

embeddings = YandexEmbeddings(
    folder_id=os.environ['YANDEX_FOLDER_ID'],
    api_key=os.environ['YANDEX_API_KEY']
)

vec = embeddings.embed_query("Hello, world!")
len(vec)

256

In [3]:
import unicodedata

ACCENT_MAPPING = {
    '́': '',
    '̀': '',
    'а́': 'а',
    'а̀': 'а',
    'е́': 'е',
    'ѐ': 'е',
    'и́': 'и',
    'ѝ': 'и',
    'о́': 'о',
    'о̀': 'о',
    'у́': 'у',
    'у̀': 'у',
    'ы́': 'ы',
    'ы̀': 'ы',
    'э́': 'э',
    'э̀': 'э',
    'ю́': 'ю',
    '̀ю': 'ю',
    'я́́': 'я',
    'я̀': 'я',
}
ACCENT_MAPPING = {unicodedata.normalize('NFKC', i): j for i, j in ACCENT_MAPPING.items()}


def unaccentify(s):
    source = unicodedata.normalize('NFKC', s)
    for old, new in ACCENT_MAPPING.items():
        source = source.replace(old, new)
    return source

def normalize(text):
    return (unaccentify(text)
            .replace('«','')
            .replace('»','')
            .replace('"','')
            .replace('<','')
            .replace('>',''))


In [9]:
with open('../source/dataset.txt',encoding='utf-8') as f:
    dataset = ''.join(f.readlines())

dataset = [normalize(x)[:2900] for x in dataset.split('-----')]

In [10]:
max(len(x) for x in dataset)

2900

In [11]:
from langchain_chroma import Chroma

db = Chroma.from_texts(dataset, embeddings, persist_directory='./chroma_db')

In [12]:
from langchain_chroma import Chroma
db = Chroma(embedding_function=embeddings, persist_directory='./chroma_db')

In [13]:
q = "Аневризма дуги аорты"
retriever = db.as_retriever(search_type="mmr", search_kwargs={"k": 5})
res = retriever.invoke(q)
res

[Document(page_content='\nВ случае распространения дистальной зоны аневризмы на дугу аорты исчезает зона “шейки” для пережатия аорты в неизмененной области, и должны выполняться открытый дистальный анастомоз с дугой аорты или ее протезирование по типу полудуги (hemiarch).\n'),
 Document(page_content='\nЭндолики возникали в 8,1%, и в 7,8% развивались аневризмы дистального отдела аорты или сохранялся кровоток в ЛП с расширением аневризмы.\n'),
 Document(page_content='\nЕсли диаметр аорты превышает 45 мм или  имеет место значительное отклонение от исходных исследований, следует рассмотреть возможность более частой визуализации  I, A 1, A У взрослых родственников первой степени пациентов с бикуспидальным аортальным клапаном, синдром ными аневризмами аорты  рекомендуется визуализацию дуги аорты для выявления бессимптомных носителей  I, B 2, B Рекомендуется проведение генетического тестирования родственников первой степени пациентов с наличием мутации гена, ассоциированного с аневризмой груд

Для получения естественного ответа на вопрос необходимо построить цепочку, которая сначала вызывает `retriever` для извлечения релевантных фрагментов текста, а затем забрасывает релевантные фрагменты в контекст модели, и просит её ответить на исходный вопрос с учётом контекста.

In [15]:
import langchain.chains
import langchain.prompts
from yandex_chain import YandexLLM, YandexGPTModel
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

llm = YandexLLM(folder_id=os.environ['YANDEX_FOLDER_ID'],
                api_key=os.environ['YANDEX_API_KEY'],
                model=YandexGPTModel.Pro)

prompt = """
Ты - кардиохирург. Пожалуйста, посмотри на текст и симптомы больного ниже. Напиши краткое заключение с рекомендацией по лечению для больного.
Текст:
-----
{context}
-----
Симптомы:
{simptoms}
Заключение:"""

prompt = langchain.prompts.PromptTemplate(
    template=prompt, input_variables=["context", "simptoms"]
)

def join_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Создаём цепочку
chain = (
    {"context": retriever | join_docs, "simptoms": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

chain.invoke(q)

'На основании предоставленной информации и с учётом симптомов аневризмы дуги аорты, можно предположить, что в данном случае требуется хирургическое вмешательство.\n\nРекомендуется проведение детального обследования для оценки состояния аорты и определения дальнейшей стратегии лечения. В зависимости от результатов обследования, может быть предложено открытое хирургическое лечение (дистальный анастомоз с дугой аорты или её протезирование по типу полудуги), либо другие методы лечения аневризмы. \n\nБольному следует отказаться от курения и строго следовать рекомендациям врача. Необходимо регулярно проходить контрольные обследования и следить за состоянием здоровья. \n\nОкончательное решение о методе лечения должен принимать квалифицированный врач после тщательного анализа всех аспектов конкретной ситуации пациента.'

In [16]:
chain.invoke('Аневризма инфраренального и интерренального отделов брюшной аорты диаметром более 4,5 см у женщин и более 5,0 см у мужчин')

'На основе предоставленной информации и учитывая, что у пациента диагностирована аневризма инфраренального и интерренального отделов брюшной аорты диаметром более 4,5 см у женщин или 5,0 см у мужчин, можно сделать вывод о необходимости проведения дальнейшей диагностики и оценки состояния пациента.\n\nУчитывая размер аневризмы, рекомендуется провести дополнительные исследования для оценки прогрессирования заболевания и возможных рисков. Важно выполнить полную визуализацию аорты и при необходимости рассмотреть возможность генетического тестирования. \n\nПоскольку аневризма имеет диаметр более 4,5 см (у женщин) или 5,0 см (у мужчин), это может указывать на симптомное течение болезни. В этом случае пациенту следует назначить медикаментозное противовоспалительное лечение.\n\nНеобходимо провести консультации с профильными специалистами для определения оптимальной стратегии лечения и наблюдения за состоянием пациента.'

## Hypothetical questions RAG

In [18]:
extract_q_prompt = """
## Задача
Ты - кардиохирург, задача которого - выделить из текста как можно больше симптомов, информации о пациенте и о лечении. 
На вход поступает фрагмент текста, твоя задача - вернуть список симптомов, информации о пациенте, лечении или вопросов к тесту. Каждый элемент списка пиши в скобках с новой строки. Не пиши никакого другого текста, кроме симптомов, информации о пациенте, лечении или вопросов. Старайся избегать местоимений, по возможности пиши более конкретные предложения.

## Пример:
Текст: Хирургическое  лечение мужчинам  с АБА рекомендуется при диаметре  аневризмы  55,5 см у мужчин 27  У женщин с  АБА и  приемлемым хирургичес ким риском оперативное вмешательство может быть рекомендовано при диаметре АБА ≥ 4,55 см
Результат:
(Мужчина с АБА, диаметр аневризмы 55,5 см)
(Женщина с АБА, диаметр аневризмы ≥ 4,55 см)
(Хирургическое лечение)
(Оперативное вмешательство)

## Задание
Текст: {}
Результат:
"""

llm.invoke(extract_q_prompt.format(dataset[0]))

'(Наблюдение за аневризмами)\n(Ультразвуковое исследование)\n(Аневризмы диаметром 3 – 3,9 см — один раз в три года)\n(Аневризмы диаметром 4,0 – 4,9 см — один раз в год)\n(Аневризмы ≥ 5,0 см — один раз в 3–6 месяцев)'

In [19]:
import json
import re
from tqdm.auto import tqdm


questions = []


def extract_questions(text):
    res = llm.invoke(extract_q_prompt.format(text))
    res = res.split('\n')
    q = []

    for x in res:
        if z := re.match(r'\((.*)\)', x):
            z = z.string.strip()[1:-1]
            q.append(z)

    return q


for text in tqdm(dataset):
    q = extract_questions(text)
    questions.append({
        "text": text,
        "questions": q,
    })

with open('hypo_questions/questions.json', 'w', encoding='utf-8') as f:
    json.dump(questions, f, indent=4, ensure_ascii=False)

  0%|          | 0/473 [00:00<?, ?it/s]

Загрузим гипотетические вопросы с диска.

In [20]:
with open('hypo_questions/questions.json', encoding='utf-8') as f:
    questions = json.load(f)

In [21]:
qdb = Chroma(
    embedding_function=embeddings,
    persist_directory='./chroma_hypoq_db',
)

for x in questions:
    if len(x['questions']) > 0:
        meta = [{"text": x['text']}] * len(x['questions'])
        qdb.add_texts(x['questions'], metadatas=meta)

In [22]:
qdb = Chroma(
    embedding_function=embeddings,
    persist_directory='./chroma_hypoq_db',
)

In [23]:
q = "Мужчина, АБА, диаметр 55,5 см"
retriever = qdb.as_retriever(search_type="mmr", search_kwargs={"k": 10})
res = retriever.invoke(q)
res

[Document(metadata={'text': '\nХирургическое  лечение мужчинам  с АБА рекомендуется при диаметре  аневризмы  55,5 см у мужчин 27 \uf0b7 У женщин с  АБА и  приемлемым хирургичес ким риском оперативное вмешательство может быть рекомендовано при диаметре АБА ≥ 4,55,0 см; \uf0b7 При быстром росте АБА – более 1,0 см в год рекомендовано срочное обращение к сосудистому хирургу для дополнительного обследования. УДД 5  УУР С \n'}, page_content='Мужчина с АБА, диаметр аневризмы 55,5 см'),
 Document(metadata={'text': '\nХирургическая коррекция расширения восходящей аорты рекомендуется во всех случаях при аневризме корня и/или тубулярной части восходящей аорты и ее максимальном диаметре 55 мм вне зависимости от выраженности АН\n'}, page_content='Максимальный диаметр аорты 55 мм'),
 Document(metadata={'text': '\nПациентам с АБА рекомендуется приём малых доз ацети лсалициловой кислоты ** , прием которой должен продолжаться и в послеперационном пе риоде.   .\n'}, page_content='Пациент с АБА'),
 Docum

В нашем случае нам нужен более хитрый алгоритм, при котором мы возвращаем не сами найденные вопросы, а соответствующие им тексты. Кроме того, надо учесть, что тексты могут повторяться, поэтому с помощью `set` выберем из них уникальные:

In [24]:
def xretriever(q):
    res = retriever.invoke(q)
    return '\n'.join(set([x.metadata['text'] for x in res]))

print(xretriever(q))


84 Уровень доказательности В качестве  визуализирующего метода исследования первой линии при патологии дуги аорты рекомендуется проведение КТ ангиографии  I, C 2, C Рекомендуется выполнять измерение диаметров аорты на основании заранее заданных анатомических ориентиров перпендикулярно продольной оси  I, C 3, C При повторных визуализирующих исследованиях аорты в течение определенного времени для оценки изменений диаметра рекомендуется использ овать метод визуализации с самым низким риском ятрогенных осложнений  I, C 2, C Для предоперационного планирования следует визуализировать всю грудную аорту и аортальный клапан с помощью КТА или МРТ  IIa, B 3, B ЧП-ЭхоКГ может быть подходящей альтернативой  при нестабильной гемодинамике, невозможности транспортировки пациента, неясных результатах МРТ или КТА  IIa, B 3, B ЧП-ЭхоКГ рекомендуется во время всех открытых операций на дуге аорте  I, B 2, B


Наоборот, недавний систематический анализ не показал статистически значимого увеличения риска сме

In [26]:
chain = (
    {"context": xretriever, "simptoms": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

chain.invoke(q)

'На основе предоставленных данных можно сделать следующее заключение: пациенту с аневризмой брюшной аорты (АБА) диаметром 55,5 см рекомендуется хирургическое лечение. Это необходимо для предотвращения её разрыва и возможных осложнений.\n\nПеред операцией необходимо провести полную визуализацию всей грудной аорты и аортального клапана с помощью КТА или МРТ. Если состояние пациента нестабильно или его нельзя транспортировать, то альтернативой может стать чреспищеводная эхокардиография (ЧП-ЭхоКГ).\n\nПациентам с АБА также рекомендуется принимать малые дозы ацетилсалициловой кислоты до и после операции. \n\nДля плановой открытой хирургии АБА следует выбирать стационары с большим опытом проведения таких операций (не менее 10–20 в год). При меньшем объёме операций в учреждении предпочтительно направить пациента в специализированный областной или федеральный центр.'

In [27]:
chain.invoke('Аневризма инфраренального и интерренального отделов брюшной аорты диаметром более 4,5 см у женщин и более 5,0 см у мужчин')

'Учитывая, что диаметр аневризмы инфраренального и интерренального отделов брюшной аорты больше 4,5 см у женщин или 5,0 см у мужчин, существует риск её разрыва. Рекомендую обратиться к сосудистому хирургу для проведения дополнительной визуализации и назначения операции по протезированию аневризмы брюшной аорты. Это необходимо для предотвращения разрыва аневризмы и снижения риска возможных осложнений.'

In [30]:
chain.invoke('Женщина с диаметром аневризмы 5 см')

'Учитывая размер аневризмы (5 см), пациентке рекомендуется хирургическое вмешательство с целью предотвращения разрыва аневризмы. \n\nЖенщинам с аневризмой брюшной аорты диаметром ≥ 4,5 см хирургическое лечение показано. Перед операцией необходимо провести дополнительное обследование и проконсультироваться с сосудистым хирургом.'

In [31]:
chain.invoke('Мужчина, эндоподтекание типа I')

'На основе предоставленной информации, пациенту с эндоподтеканием типа I после эндоваскулярной реконструкции аневризмы брюшной аорты рекомендуется провести оценку необходимости повторного вмешательства для достижения герметизации, преимущественно эндоваскулярными методами. \n\nТакже необходимо контролировать состояние пациента через регулярное проведение КТ-ангиографии через 1 месяц и 12 месяцев после операции, а затем ежегодно в течение жизни или чаще, если будут выявлены признаки беспокойства или эндоподтекания после 1 месяца наблюдения. \n\nРекомендуется обсудить индивидуальную программу диагностики и лечения с лечащим врачом для оценки состояния здоровья и выбора оптимального подхода к лечению.'

In [33]:
chain.invoke('рАБА, тяжелое состояние')

'На основании предоставленных данных и учитывая тяжёлое состояние пациента с разорвавшейся аневризмой брюшной аорты (рАБА), рекомендуется экстренное хирургическое вмешательство (открытая операция). Пациента следует немедленно госпитализировать в медицинское учреждение с круглосуточным сосудистым отделением.'

In [36]:
chain.invoke('Жалобы на одышку при физической нагрузке, приступ удушья')

'На основании предоставленных данных и с учётом наличия одышки при физической нагрузке и приступов удушья у пациента, можно сделать следующее заключение:\n\nДля определения состояния сердца и лёгких, а также для оценки функциональной способности пациента перед плановой реконструкцией аневризмы брюшной аорты рекомендуется провести кардиопульмональные нагрузочные тесты и исследование функции внешнего дыхания (спирометрию). При необходимости следует назначить лечение или дополнительные обследования у специалистов.\n\n**Важно!** Предоставленные рекомендации основаны на анализе имеющихся данных и являются гипотетическими. Окончательное решение о необходимых мерах принимает врач на основании полной информации о пациенте после личного осмотра и дополнительных исследований.'

In [35]:
chain.invoke('Женщина, нарушен кровоток в малом тазу')

'На основании предоставленных данных, женщине с нарушением кровотока в малом тазу рекомендуется провести детальную диагностику для определения причины нарушения кровотока. Учитывая важность сохранения дистального коллатерального кровообращения в малом тазу, необходимо рассмотреть методы визуализации, такие как КТ с контрастированием. \n\nВ случае выявления патологии, такой как аневризма или другие сосудистые проблемы, может потребоваться консультация со специалистом по эндоваскулярным процедурам или кардиохирургом для оценки возможности проведения хирургического вмешательства, например, эмболизации или лигирования внутренней подвздошной артерии, либо других процедур для восстановления нормального кровотока.\n\nТакже необходимо контролировать артериальное давление и общее состояние пациентки для предотвращения возможных осложнений. Важно следовать рекомендациям врача и проходить регулярные обследования для своевременного выявления и лечения возможных проблем.'